# Modern AI Pro: Simple Chatbot on your enterprise content

We will scrape the web content as well as our PDFs to answer user queries more effectively.


In [ ]:
!pip install --upgrade --quiet  sentence-transformers gradio bs4 pypdf2 chromadb rapidocr-onnxruntime langchain langchain-community langchainhub langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 59.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 38.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.9/14.9 MB 78.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.2/806.2 kB 63.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 80.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 13.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.9/307.9 kB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 10

## Step 1: Scraping our web content and using that as the foundation.


In [ ]:
# Load, chunk and index the contents of a blog/website.
from langchain_community.document_loaders import WebBaseLoader
import bs4
loader = WebBaseLoader(
    web_paths=("https://mitrarobot.com",)
)
docs = loader.load()
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [ ]:
# Split and manage the texts
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

In [ ]:
# Store embeddings in a Vector DB
from langchain_community.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2", model_kwargs={"device": "cpu"})
vectorstore_web = Chroma.from_documents(documents=splits, embedding=embeddings)
retriever_web = vectorstore_web.as_retriever()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
from google.colab import userdata

!pip install git+https://gitlab.com/gauthammsam/mitrallm.git --quiet
from mitrallm import MitraLLM
llm = MitraLLM(
    token     = userdata.get("MITRA_TOKEN"),
    accessapi = userdata.get("MITRA_ENDPOINT")
)

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.0/817.0 kB 11.0 MB/s eta 0:00:00


In [ ]:
# Retrieve and generate using the relevant snippets of the blog.
from langchain import hub
from langchain_openai import ChatOpenAI
prompt = hub.pull("rlm/rag-prompt")

In [ ]:
# Building the RAG
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
rag_web = (
    {"context": retriever_web | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
rag_web.invoke("What are the features of the Mitra robot?")

'The Mitra robot features pattern recognition with computer vision tools, autonomous navigation with obstacle'

## Step 2: Extracting content from PDFs
Get one of our [research papers](https://drive.google.com/file/d/1kfjF9iuGG74ORFGu4v9dMO15pgsKI5Rh/view?usp=sharing) for sample. Download a copy locally and upload to the runtime.

In [ ]:
from PyPDF2 import PdfReader
reader = PdfReader('arso.pdf')
text = ""
for i in range(0, len(reader.pages)):
    page = reader.pages[i]
    text += page.extract_text() + " "

In [ ]:
from langchain.schema.document import Document

documents = [Document(page_content=text, metadata={"source": "local"})]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=40)
all_splits = text_splitter.split_documents(documents)

In [ ]:
vectordb_paper = Chroma.from_documents(documents=all_splits, embedding=embeddings, persist_directory="chroma_db_paper")
retriever_paper = vectordb_paper.as_retriever()

In [ ]:
all_splits

In [ ]:
# Building the RAG
rag_pdf = (
    {"context": retriever_paper | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
rag_pdf.invoke("Tell me about the likability index")

'The likability index refers to a survey that measures how end-users perceive different robots'

# Let's now build a chatbot with this
You will need Gradio for this.

In [ ]:
import gradio as gr


In [ ]:
def respond(input_text, option):
    # Simple response logic (you can expand this with real chatbot logic)
    if option == 'PDF':
        response = rag_pdf.invoke(input_text)
    else:
        response = rag_web.invoke(input_text)

    return response

with gr.Blocks() as app:
    gr.Markdown("## Chat Mitra")
    with gr.Row():
        input_text = gr.Textbox(label="Your Input")
        option = gr.Radio(choices=['PDF', 'Web'], label="Choose an Option", value='PDF')
        submit_button = gr.Button("Submit")
    output_text = gr.Textbox(label="Chatbot Response")

    submit_button.click(fn=respond, inputs=[input_text, option], outputs=output_text)

app.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://07ed9733a1267c26a9.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
